# 1.2 Download Artigos SBF

## Imports

In [1]:
import os
from urllib.parse import urljoin

from bs4 import BeautifulSoup, SoupStrainer
from tqdm import tqdm

from functions import Request

In [2]:
request = Request()

## Funções

In [25]:
sbf_link = lambda x: urljoin('http://www.sbfisica.org.br/rbef/', x)

In [26]:
def volumes_por_ano() -> dict[int:list]:
    req = request.get('http://www.sbfisica.org.br/rbef/edicoes.shtml')
    soup = BeautifulSoup(req.text, 'html.parser', parse_only=SoupStrainer('table', attrs={'id':'AutoNumber2'}))
    links = {}
    
    for row in soup.find_all('tr', attrs={'bordercolor': '#FFFFFF'}):
        linha = row.find_all('td')
        
        if len(linha) == 6:  # (ano, volume, link1, link2, link3, link4)
            ano = int(linha[0].text) 
            links[ano] = []
            
            for i in range(2,6):  # do elemento 2 até o elemento 5, são as colunas que possuem urls.
                a_tag = linha[i].find('a')
                if a_tag:
                    links[ano].append(sbf_link(a_tag['href']))
                    
    return links

## Código

In [69]:
volumes = volumes_por_ano()

In [72]:
try:
    os.mkdir(os.path.join(os.getcwd(), 'artigos_brutos')) # cria a pasta dos artigos brutos
except FileExistsError:
    print('A pasta `artigos_brutos` já existe!')

try:
    os.mkdir(os.path.join(os.getcwd(), 'artigos_brutos', 'SBF')) # cria a pasta da SBF dentro dos artigos brutos
except FileExistsError:
    print('A pasta `artigos_brutos\SBF` já existe!')

A pasta `artigos_brutos` já existe!
A pasta `artigos_brutos\SBF` já existe!


In [106]:
for ano, vol in tqdm(volumes.items()):
    dir_path = os.path.join(os.getcwd(), 'artigos_brutos', 'SBF', str(ano))
    
    try:
        os.mkdir(dir_path)
    except FileExistsError:
        pass
    
    for link in vol:
        req = request.get(link)
        soup = BeautifulSoup(req.text, 'html.parser', parse_only=SoupStrainer('table'))
        
        seção = 'Sem Categoria'
        try:
            os.mkdir(os.path.join(dir_path, seção))
        except FileExistsError:
            pass
        
        for row in soup.find_all('tr'):
            if row.span:
                classe = row.span['class']
                
                if 'titulosessao' in classe:
                    seção = row.span.text.strip()
                    if row.span.br:
                        seção = next(row.span.stripped_strings)
                        
                    seção = seção.replace(':', '').replace('/', '').replace('?', '')
                    try:
                        os.mkdir(os.path.join(dir_path, seção))
                    except FileExistsError:
                        pass
            
                elif 'titulo' in classe:
                    url_arquivo = sbf_link(row.span.a['href'])
                    req = request.get(url_arquivo)
                    nome = row.span.a.text.replace(':', '').replace('?', '').replace('/', '').replace('\r', '').replace('"', '')
                    
                    if len(nome) > 150:
                        nome = nome[:150]
                                            
                    nome += '.pdf'
                    
                    with open(os.path.join(dir_path, seção, nome), 'wb') as file:
                        file.write(req.content)

100%|██████████| 23/23 [02:17<00:00,  5.97s/it]
